# Simulación de las llegadas de las llamadas telefónicas
Las llamadas siguen una distribución de Poisson con una tasa media de 20 llamadas por hora. Se desea simular el número de llamadas que se reciben en un intervalo de X horas.

In [24]:
import random
import math
def generar_poisson(media):
    """
    Función para generar un número aleatorio a partir de una distribución de Poisson
    Formula: P(k) = (e^-λ * λ^k) / k!
    """
    k = 0
    p = math.exp(-media)
    acumulado = p
    r = random.random()
    while acumulado < r:
      k += 1
      p = p * media / k
      acumulado += p
    return k

def generar_dist_acumulado(dist, media, n):
    """
    Función para generar n números aleatorios a partir de una distribución de Poisson
    """
    return [dist(media) for _ in range(n)]


In [33]:
def generar_exponencial(media):
  """
  Función para generar un número aleatorio a partir de una distribución exponencial
  """
  u = random.random()
  return -media * math.log(1 - u)


In [78]:
import matplotlib.pyplot as plt
# import numpy as np
llamadas = []
for i in range (1000):
    llamadas.append(generar_exponencial(3))

tiempo = 0
for x in range(0, 1000):
    tiempo += llamadas[x]
    if tiempo > 60:
        print(x)
        break

# comprobar que el promedio por hora es 20

17


In [79]:
import simpy
import math
class Call:
    def __init__(self, env, central, time, name):
        self.env = env
        self.central = central
        self.name = name
        self.time = time
    def makeCall(self):
        yield self.env.timeout(self.time)
    def action(self):
        print(f"{self.name} solicita una llamada en {self.env.now}")
        if self.central.count < self.central.capacity:
            with self.central.request() as req:
                yield req
                print(f"{self.name} se comunica con el operador {self.central.count} en {self.env.now}")
                yield self.env.process(self.makeCall())
                print(f"{self.name} finaliza la llamada en {self.env.now}")
        else:
            print(f"Se pierde la llamada de {self.name}")
                
def setup(env, central):
    i = 0
    while True:
        t_llamada = generar_exponencial(3)
        yield env.timeout(t_llamada)
        dur_llamada = generar_exponencial(6)
        c = Call(env, central, dur_llamada, name=f"Cliente {i}")
        env.process(c.action())
        i +=1

env = simpy.Environment()
central = simpy.Resource(env, capacity=3)
env.process(setup(env,central))

env.run(until=180)
    

Cliente 0 solicita una llamada en 3.4801223866500974
Cliente 0 se comunica con el operador 1 en 3.4801223866500974
Cliente 1 solicita una llamada en 9.81355585028086
Cliente 1 se comunica con el operador 2 en 9.81355585028086
Cliente 1 finaliza la llamada en 11.148523344730119
Cliente 0 finaliza la llamada en 12.927053034726875
Cliente 2 solicita una llamada en 13.509502792852828
Cliente 2 se comunica con el operador 1 en 13.509502792852828
Cliente 3 solicita una llamada en 14.692479938682876
Cliente 3 se comunica con el operador 2 en 14.692479938682876
Cliente 4 solicita una llamada en 15.892183363985698
Cliente 4 se comunica con el operador 3 en 15.892183363985698
Cliente 2 finaliza la llamada en 16.227771601104614
Cliente 5 solicita una llamada en 16.545412102786294
Cliente 5 se comunica con el operador 3 en 16.545412102786294
Cliente 6 solicita una llamada en 16.976581076131744
Se pierde la llamada de Cliente 6
Cliente 3 finaliza la llamada en 17.868917178160718
Cliente 4 finaliza 